In [1]:
# let's introduce Message types and a simple messaging queue for communication between nodes.

In [2]:
class Message:
    def __init__(self, sender, receiver, content):
        self.sender = sender
        self.receiver = receiver
        self.content = content

In [3]:
class BlockProposal(Message):
    pass

In [4]:
class Prevote(Message):
    pass

In [5]:
class Precommit(Message):
    pass

In [7]:
class Block:
    def __init__(self, data):
        self.data = data

In [11]:
class Node:
    def __init__(self, name):
        self.name = name
        self.round = 0
        self.step = 0
        self.block_proposal = None
        self.prevotes = []
        self.precommits = []

    def propose(self, block):
        # A proposer broadcasts a proposal to all other nodes.
        for node in nodes:
            if node != self:
                messaging_queue.append(BlockProposal(self, node, block))

    def receive_message(self):
        # Nodes can receive messages from the queue.
        for message in list(messaging_queue):  # We make a copy of the list so we can modify it while iterating
            if message.receiver == self:
                if isinstance(message, BlockProposal) and self.step == 0:
                    self.block_proposal = message.content
                    self.step = 1
                    # The node prevotes for the block as soon as it sees it.
                    for node in nodes:
                        if node != self:
                            messaging_queue.append(Prevote(self, node, self.block_proposal))
                elif isinstance(message, Prevote) and self.step == 1:
                    self.prevotes.append(message.content)
                    # If a node receives more than 2/3 prevotes for a block, it precommits to it.
                    if self.prevotes.count(self.block_proposal) > 2/3*len(nodes):
                        self.step = 2
                        for node in nodes:
                            if node != self:
                                messaging_queue.append(Precommit(self, node, self.block_proposal))
                elif isinstance(message, Precommit) and self.step == 2:
                    self.precommits.append(message.content)
                    # If a node receives more than 2/3 precommits for a block, it commits it.
                    if self.precommits.count(self.block_proposal) > 2/3*len(nodes):
                        self.step = 3
                        print(f'{self.name} commits block {self.block_proposal.data}')
                messaging_queue.remove(message)

In [12]:
messaging_queue = []

In [10]:
block = Block('Block1')
proposer = nodes[0]
proposer.propose(block)

NameError: name 'Block' is not defined